# 3.3: Split-apply-combine in pandas

Now that we are (somewhat) comfortable with transformation of data between wide and long, we can get into another very powerful pandas feature known as split-apply-combine.

---

## Dataset

We will again be using the dataset measuring "Machiavellianism" in people.

The three files are in your ```../assets/datasets/``` directory. They are:

- ```mach_data.csv``` which contains the wide data.
- ```mach_long.csv``` which contains the already long data.
- ```mach_codebook.csv``` which contains the information about the survey data.

---

## Packages

Loaded same as ever.

In [1]:
# data modules
import numpy as np
import scipy.stats as stats
import pandas as pd

# plotting modules
import matplotlib.pyplot as plt
import seaborn as sns

# make sure charts appear in the notebook:
%matplotlib inline

---

## A: Load the already widened data

I have already widened the data for you here in the interest of time. You can see how I get started on the bonus 2 question below as well.

If you would like to do the melting part yourself for practice, be my guest! The more practice the better. If doing it yourself you would instead load the ```mach_data.csv``` file again.

In [2]:
mach_long = pd.read_csv('../assets/datasets/mach_long.csv')
mach_long.head()

,age,gender,subject_id,variable,value
0,24,1,1,Q1,4
1,33,2,2,Q1,2
2,21,1,3,Q1,3
3,17,1,4,Q1,4
4,22,1,5,Q1,4


In [3]:
print mach_long.variable.unique()

['Q1' 'Q2' 'Q3' 'Q4' 'Q5' 'Q6' 'Q7' 'Q8' 'Q9' 'Q10' 'Q11' 'Q12' 'Q13' 'Q14'
 'Q15' 'Q16' 'Q17' 'Q18' 'Q19' 'Q20' 'score' 'seconds_elapsed']


In [4]:
mach_long = mach_long[mach_long.variable != 'score']

seconds_data = mach_long.ix[mach_long.variable == 'seconds_elapsed', ['subject_id','variable','value']]
seconds_data['seconds_per_q'] = seconds_data.value / 20.

mach_long = mach_long[mach_long.variable != 'seconds_elapsed']

seconds_data.head()

,subject_id,variable,value,seconds_per_q
255906,1,seconds_elapsed,177,8.85
255907,2,seconds_elapsed,107,5.35
255908,3,seconds_elapsed,323,16.15
255909,4,seconds_elapsed,136,6.80
255910,5,seconds_elapsed,142,7.10


### A.2 : Sorting by multiple columns with custom sort for questions

We are going to use ```pd.Categorical``` to custom-sort the questions after subject

see:  http://stackoverflow.com/questions/13838405/custom-sorting-in-pandas-dataframe

Look stuff up online!!

In [5]:
categorical_questions = pd.Categorical(mach_long.variable,
                                       ['Q1','Q2','Q3','Q4',
                                        'Q5','Q6','Q7','Q8',
                                        'Q9','Q10','Q11','Q12',
                                        'Q13','Q14','Q15','Q16',
                                        'Q17','Q18','Q19','Q20'])

mach_long['variable'] = categorical_questions

mach_long.sort_values(['subject_id','variable'], inplace=True)

mach_long.head()

,age,gender,subject_id,variable,value
0,24,1,1,Q1,4
12186,24,1,1,Q2,4
24372,24,1,1,Q3,2
36558,24,1,1,Q4,2
48744,24,1,1,Q5,4


### A.3: Adding the time by question by subject_id as a new value

Merge the seconds data and the long data together. This is a preview to future lessons!

In [6]:
mach_secs = mach_long.merge(seconds_data[['subject_id', 'seconds_per_q']], on='subject_id', sort=False)

mach_secs.head()

,age,gender,subject_id,variable,value,seconds_per_q
0,24,1,1,Q1,4,8.85
1,24,1,1,Q2,4,8.85
2,24,1,1,Q3,2,8.85
3,24,1,1,Q4,2,8.85
4,24,1,1,Q5,4,8.85


### A.4 Doing a cumulative sum of the seconds

Do a cumulative sum of the seconds by subject.

This is an example of the split-apply-combine pattern. We will do more as well.

#### A.4.1: Split the data into groups by subject_id

In [7]:
# The groupby iterator
mach_subject_groups = mach_secs.groupby(by=['subject_id'])

#### A.4.2: Apply the cumulative sum function, iterating through the groups

the ```.transform()``` function on group objects requires a function as the argument.

In [8]:
def secs_cumulative_sum(seconds_per_q):
    cum_seconds = seconds_per_q.cumsum()
    return cum_seconds

mach_secs['seconds_elapsed'] = mach_subject_groups.seconds_per_q.transform(secs_cumulative_sum)

In [9]:
mach_secs.head()

,age,gender,subject_id,variable,value,seconds_per_q,seconds_elapsed
0,24,1,1,Q1,4,8.85,8.85
1,24,1,1,Q2,4,8.85,17.70
2,24,1,1,Q3,2,8.85,26.55
3,24,1,1,Q4,2,8.85,35.40
4,24,1,1,Q5,4,8.85,44.25


### A.3: Make the re-widened dataset with TWO value columns

This is where the hierarchical index pattern really shines.

In [10]:
mach_secs_wide = pd.pivot_table(mach_secs,
                                values=['value','seconds_elapsed'],
                                index=['subject_id','age','gender'],
                                columns=['variable'])

In [11]:
mach_secs_wide.head()

value                                               \
variable                 Q1  Q10  Q11  Q12  Q13  Q14  Q15  Q16  Q17  Q18   
subject_id age gender                                                      
1          24  1        4.0  4.0  2.0  5.0  2.0  1.0  4.0  4.0  2.0  0.0   
2          33  2        2.0  4.0  2.0  4.0  2.0  2.0  3.0  2.0  2.0  4.0   
3          21  1        3.0  3.0  2.0  4.0  4.0  3.0  3.0  4.0  2.0  4.0   
4          17  1        4.0  3.0  1.0  5.0  3.0  2.0  4.0  2.0  2.0  3.0   
5          22  1        4.0  3.0  1.0  2.0  4.0  4.0  5.0  4.0  1.0  4.0   

                        ...   seconds_elapsed                            \
variable                ...               Q19    Q2    Q20     Q3    Q4   
subject_id age gender   ...                                               
1          24  1        ...            168.15  17.7  177.0  26.55  35.4   
2          33  2        ...            101.65  10.7  107.0  16.05  21.4   
3          21  1        ...            306.85  32.3  323.0  48.45  64.6   
4          17  1        ...            129.20  13.6  136.0  20.40  27.2   
5          22  1        ...            134.90  14.2  142.0  21.30  28.4   

                                                           
variable                  Q5    Q6      Q7     Q8      Q9  
subject_id age gender                                      
1          24  1       44.25  53.1   61.95   70.8   79.65  
2          33  2       26.75  32.1   37.45   42.8   48.15  
3          21  1       80.75  96.9  113.05  129.2  145.35  
4          17  1       34.00  40.8   47.60   54.4   61.20  
5          22  1       35.50  42.6   49.70   56.8   63.90  

[5 rows x 40 columns]

### A.4: Indexing on the MultiIndex DataFrame

#### A.4.1 create a ```pd.IndexSlice``` object

In [12]:
idx = pd.IndexSlice

#### A.4.2 Pull out the seconds elapsed at questions 2 and 5 for subjects over 70 years old

In [13]:
print mach_secs_wide.index.names
print mach_secs_wide.columns.names
#mach_secs_wide.loc[idx['age',:,:]]

[u'subject_id', u'age', u'gender']
[None, u'variable']


In [32]:
mach_secs_wide.loc[idx[:,70:,:],idx[:,['seconds_elapsed','Q1']]]

,,,value,seconds_elapsed
,,variable,Q1,Q1
subject_id,age,gender,,
657,80,1,4.0,8.50
740,74,2,2.0,16.35
1662,77,1,4.0,9.95
2050,72,1,2.0,13.40
2087,70,1,4.0,6.05
2538,73,1,3.0,7.55
2569,71,1,2.0,11.25
2580,81,1,2.0,11.40
